# Noveum Platform API Client - Complete Demo

This notebook demonstrates all 28 methods available in the Noveum Platform API Client.

## API Methods Coverage:
- **Trace Methods (7)**: Connection status, directory tree, ingest, query, get trace, get spans
- **Dataset Methods (15)**: Create, list, get, update, delete, versions, version diff, items management
- **Scorer Results Methods (6)**: Create, batch create, list, get, update, delete

## Prerequisites:
- Set `NOVEUM_API_KEY` in your environment or `.env` file
- Optionally set `NOVEUM_ORGANIZATION_ID`


## Setup


In [ ]:
# Cell 0: Import required libraries
import json
from pprint import pprint

from dotenv import load_dotenv

from novaeval.noveum_platform import NoveumClient

# Load environment variables
load_dotenv()

print("✓ Libraries imported successfully")


In [ ]:
# Cell 1: Configuration Variables
# Customize these values as needed

PROJECT_NAME = "noveum-api-wrapper-demo"
ENVIRONMENT = "development"
DATASET_NAME = "Sample Traces Wrapper Demo 9"
DATASET_SLUG = "reddit-traces-wrapper-demo-9"
DATASET_VERSION = "1.0.0"
SCORER_ID = "demo-quality-scorer"
ORGANIZATION_SLUG = "magic-api"  # Adjust if needed
NUM_TRACES = 10

print("Configuration:")
print(f"  Project: {PROJECT_NAME}")
print(f"  Environment: {ENVIRONMENT}")
print(f"  Dataset: {DATASET_NAME}")
print(f"  Dataset Slug: {DATASET_SLUG}")
print(f"  Version: {DATASET_VERSION}")
print(f"  Scorer ID: {SCORER_ID}")
print(f"  Number of traces: {NUM_TRACES}")
print(f"  Organization Slug: {ORGANIZATION_SLUG}")


In [ ]:
# Cell 2: Load Test Data
print("Loading test data from sample_traces.json...")

with open("../test_data/sample_traces.json") as f:
    all_traces = json.load(f)

# Extract first NUM_TRACES traces
traces_sample = all_traces[:NUM_TRACES]

print(f"\n✓ Loaded {len(traces_sample)} traces for demonstration")
print("\nSample trace structure (first trace):")
print(f"  Trace ID: {traces_sample[0].get('trace_id', 'N/A')}")
print(f"  Span ID: {traces_sample[0].get('span_id', 'N/A')}")
print(f"  Name: {traces_sample[0].get('name', 'N/A')}")
print(f"  Project: {traces_sample[0].get('project', 'N/A')}")
print(f"  Environment: {traces_sample[0].get('environment', 'N/A')}")
print(f"  Status: {traces_sample[0].get('status', 'N/A')}")


In [ ]:
# Cell 3: Initialize Noveum Client
print("Initializing Noveum Platform API Client...")

client = NoveumClient(
    # api_key is read from NOVEUM_API_KEY environment variable
    # organization_id is read from NOVEUM_ORGANIZATION_ID or can be passed explicitly
)

print("✓ Client initialized successfully")
print(f"  Base URL: {client.base_url}")
print(f"  Organization ID: {client.organization_id or 'Not set (will use from API key)'}")


## Trace Methods (7 methods)

Demonstrating all trace-related API methods.


In [ ]:
# Cell 4: Method 1 - get_connection_status()
print("=" * 60)
print("METHOD 1: get_connection_status()")
print("=" * 60)
print("\nChecking API connection status...\n")

connection_status = client.get_connection_status()
pprint(connection_status)


In [ ]:
# Cell 5: Method 2 - get_directory_tree()
print("=" * 60)
print("METHOD 2: get_directory_tree()")
print("=" * 60)
print("\nFetching organization directory tree...\n")

directory_tree = client.get_directory_tree()
pprint(directory_tree)


In [ ]:
# Cell 6: Method 3 - ingest_trace()
print("=" * 60)
print("METHOD 3: ingest_trace()")
print("=" * 60)
print("\nIngesting a single trace...\n")

# Prepare the first trace for ingestion
single_trace = traces_sample[0].copy()

# Update project and environment to our demo values
single_trace["project"] = PROJECT_NAME
single_trace["environment"] = ENVIRONMENT
if "trace_attributes" in single_trace:
    single_trace["trace_attributes"]["noveum.project"] = PROJECT_NAME
    single_trace["trace_attributes"]["noveum.environment"] = ENVIRONMENT

ingest_result = client.ingest_trace(single_trace)
print(f"✓ Successfully ingested trace: {single_trace['trace_id']}")
pprint(ingest_result)


In [ ]:
# Cell 7: Method 4 - ingest_traces()
print("=" * 60)
print("METHOD 4: ingest_traces()")
print("=" * 60)
print("\nIngesting multiple traces in batch...\n")

# Prepare remaining traces for batch ingestion
batch_traces = []
for trace in traces_sample[1:]:
    trace_copy = trace.copy()
    trace_copy["project"] = PROJECT_NAME
    trace_copy["environment"] = ENVIRONMENT
    if "trace_attributes" in trace_copy:
        trace_copy["trace_attributes"]["noveum.project"] = PROJECT_NAME
        trace_copy["trace_attributes"]["noveum.environment"] = ENVIRONMENT
    batch_traces.append(trace_copy)

batch_result = client.ingest_traces(batch_traces)
print(f"✓ Successfully ingested {len(batch_traces)} traces")
pprint(batch_result)


In [ ]:
# Cell 8: Method 5 - query_traces()
print("=" * 60)
print("METHOD 5: query_traces()")
print("=" * 60)
print("\nQuerying traces with filters...\n")

# Query traces with various filters
query_result = client.query_traces(
    project=PROJECT_NAME,
    environment=ENVIRONMENT,
    size=20,
    sort="start_time:desc",
    include_spans=False
)

print("Query parameters:")
print(f"  Project: {PROJECT_NAME}")
print(f"  Environment: {ENVIRONMENT}")
print("  Size: 20")
print("  Sort: start_time:desc")
print("\nQuery results:")
pprint(query_result)

# Store trace IDs for later use
if "traces" in query_result and len(query_result["traces"]) > 0:
    sample_trace_id = query_result["traces"][0].get("trace_id")
    print(f"\n✓ Found {len(query_result.get('traces', []))} traces")
    print(f"  Sample trace ID for next demos: {sample_trace_id}")
else:
    sample_trace_id = traces_sample[0]["trace_id"]
    print(f"\n! No traces in query result, using original trace ID: {sample_trace_id}")


In [ ]:
# Cell 9: Method 6 - get_trace()
print("=" * 60)
print("METHOD 6: get_trace()")
print("=" * 60)
print(f"\nFetching specific trace by ID: {sample_trace_id}\n")

trace_detail = client.get_trace(sample_trace_id)
pprint(trace_detail)


In [ ]:
# Cell 10: Method 7 - get_trace_spans()
print("=" * 60)
print("METHOD 7: get_trace_spans()")
print("=" * 60)
print(f"\nFetching spans for trace: {sample_trace_id}\n")

trace_spans = client.get_trace_spans(sample_trace_id)
pprint(trace_spans)

if "spans" in trace_spans:
    print(f"\n✓ Found {len(trace_spans['spans'])} spans")


## Dataset Methods (15 methods)

Demonstrating all dataset-related API methods.


In [ ]:
# Cell 11: Method 8 - create_dataset()
print("=" * 60)
print("METHOD 8: create_dataset()")
print("=" * 60)
print("\nCreating a new dataset...\n")

dataset = client.create_dataset(
    name=DATASET_NAME,
    slug=DATASET_SLUG,
    description="Demo dataset containing Sample traces for API demonstration",
    visibility="org",
    dataset_type="agent",
    environment=ENVIRONMENT,
    tags=["demo", "Sample", "traces"],
    custom_attributes={
        "source": "sample_traces.json",
        "purpose": "API demonstration"
    }
)

print("✓ Dataset created successfully")
pprint(dataset)


In [ ]:
# Cell 12: Method 9 - list_datasets()
print("=" * 60)
print("METHOD 9: list_datasets()")
print("=" * 60)
print("\nListing all datasets...\n")

datasets_list = client.list_datasets(
    limit=20,
    offset=0,
    visibility="org",
    includeVersions=True
)

pprint(datasets_list)

if "datasets" in datasets_list:
    print(f"\n✓ Found {len(datasets_list['datasets'])} datasets")


In [ ]:
# Cell 13: Method 10 - get_dataset()
print("=" * 60)
print("METHOD 10: get_dataset()")
print("=" * 60)
print(f"\nFetching dataset by slug: {DATASET_SLUG}\n")

dataset_detail = client.get_dataset(DATASET_SLUG)
pprint(dataset_detail)


In [ ]:
# Cell 14: Method 11 - update_dataset()
print("=" * 60)
print("METHOD 11: update_dataset()")
print("=" * 60)
print(f"\nUpdating dataset: {DATASET_SLUG}\n")

updated_dataset = client.update_dataset(
    slug=DATASET_SLUG,
    description="Updated: Demo dataset with Sample traces - now with enhanced metadata",
    tags=["demo", "Sample", "traces", "updated"],
    custom_attributes={
        "source": "sample_traces.json",
        "purpose": "API demonstration",
        "updated": True,
        "version_info": "Enhanced with additional metadata"
    }
)

print("✓ Dataset updated successfully")
pprint(updated_dataset)


In [ ]:
# Cell 15: Method 12 - create_dataset_version()
print("=" * 60)
print("METHOD 12: create_dataset_version()")
print("=" * 60)
print(f"\nCreating version {DATASET_VERSION} for dataset: {DATASET_SLUG}\n")

version_data = {
    "version": DATASET_VERSION,
    "description": "Initial version with 10 Sample trace samples",
    "tags": ["v1", "initial"],
    "custom_attributes": {
        "trace_count": NUM_TRACES,
        "created_by": "demo_script"
    }
}

version = client.create_dataset_version(DATASET_SLUG, version_data)
print(f"✓ Version {DATASET_VERSION} created successfully")
pprint(version)


In [ ]:
# Cell 16: Method 13 - list_dataset_versions()
print("=" * 60)
print("METHOD 13: list_dataset_versions()")
print("=" * 60)
print(f"\nListing all versions for dataset: {DATASET_SLUG}\n")

versions_list = client.list_dataset_versions(DATASET_SLUG)
pprint(versions_list)

if "versions" in versions_list:
    print(f"\n✓ Found {len(versions_list['versions'])} versions")


In [ ]:
# Cell 17: Method 14 - get_dataset_version()
print("=" * 60)
print("METHOD 14: get_dataset_version()")
print("=" * 60)
print(f"\nFetching version {DATASET_VERSION} for dataset: {DATASET_SLUG}\n")

version_detail = client.get_dataset_version(DATASET_SLUG, DATASET_VERSION)
pprint(version_detail)


In [ ]:
# Cell 17a: Method 15 - get_dataset_versions_diff()
print("=" * 60)
print("METHOD 15: get_dataset_versions_diff()")
print("=" * 60)
print(f"\nGetting diff between current_release and next_release for: {DATASET_SLUG}\n")

versions_diff = client.get_dataset_versions_diff(DATASET_SLUG)
pprint(versions_diff)

print("\n✓ Version diff retrieved successfully")
print("This shows changes between current_release and next_release versions")


In [ ]:
# Cell 18: Transform traces into dataset items
print("=" * 60)
print("PREPARING DATASET ITEMS")
print("=" * 60)
print("\nTransforming traces into dataset item format...\n")

dataset_items = []
for i, trace in enumerate(traces_sample):
    item = {
        "item_key": f"trace_{i+1:03d}",
        "item_type": "agent_trace",
        "content": {
            **trace,
            "trace_id": trace.get("trace_id"),
            "span_id": trace.get("span_id"),
            "name": trace.get("name"),
            "start_time": trace.get("start_time"),
            "end_time": trace.get("end_time"),
            "duration_ms": trace.get("duration_ms"),
            "status": trace.get("status"),
            "attributes": trace.get("attributes", {})
        },
        "metadata": {
            "project": trace.get("project", PROJECT_NAME),
            "environment": trace.get("environment", ENVIRONMENT),
            "item_index": i + 1
        }
    }
    dataset_items.append(item)

print(f"✓ Prepared {len(dataset_items)} dataset items")
print("\nSample item structure:")
pprint(dataset_items[0])


In [ ]:
# Cell 19: Method 16 - add_dataset_items()
print("=" * 60)
print("METHOD 16: add_dataset_items()")
print("=" * 60)
print(f"\nAdding {len(dataset_items)} items to dataset: {DATASET_SLUG}\n")
print("Note: Items are automatically added to the 'next_release' version\n")

add_items_result = client.add_dataset_items(
    dataset_slug=DATASET_SLUG,
    items=dataset_items
)

print(f"✓ Successfully added {len(dataset_items)} items")
pprint(add_items_result)


In [ ]:
# Cell 22: Method 19 - publish_dataset_version()
print("=" * 60)
print("METHOD 19: publish_dataset_version()")
print("=" * 60)

print(f"\nPublishing next_release version for dataset: {DATASET_SLUG}\n")
print("Note: Automatically publishes 'next_release' and increments version number\n")

publish_result = client.publish_dataset_version(DATASET_SLUG)
print("✓ Version published successfully")
pprint(publish_result)


In [ ]:
# Cell 20: Method 17 - list_dataset_items()
print("=" * 60)
print("METHOD 17: list_dataset_items()")
print("=" * 60)
print(f"\nListing items from dataset: {DATASET_SLUG}\n")
print("Note: New optional parameters available: item_type, search, sort_by, sort_order\n")

items_list = client.list_dataset_items(
    dataset_slug=DATASET_SLUG,
    limit=20,
    offset=0
)

pprint(items_list)

if "items" in items_list:
    print(f"\n✓ Found {len(items_list['items'])} items")
    # Store first item key for later use
    if len(items_list["items"]) > 0:
        sample_item_key = items_list["items"][0].get("item_key", "trace_001")
        sample_item_id = items_list["items"][0].get("id", items_list["items"][0].get("item_key", "trace_001"))
        print(f"  Sample item key: {sample_item_key}")
        print(f"  Sample item ID: {sample_item_id}")
else:
    sample_item_key = "trace_001"
    sample_item_id = "trace_001"


In [ ]:
# # Cell 21: Method 18 - get_dataset_item() currently doesnt work
# print("=" * 60)
# print("METHOD 18: get_dataset_item()")
# print("=" * 60)
# print(f"\nFetching specific item: {sample_item_key}\n")

# item_detail = client.get_dataset_item(DATASET_SLUG, sample_item_key)
# pprint(item_detail)


In [ ]:
# Cell 23: Method 20 - delete_dataset_item()
print("=" * 60)
print("METHOD 20: delete_dataset_item()")
print("=" * 60)
print(f"\nDeleting specific item: {sample_item_id}\n")

delete_item_result = client.delete_dataset_item(DATASET_SLUG, sample_item_id)
print(f"✓ Item {sample_item_id} deleted successfully")
pprint(delete_item_result)


In [ ]:
# Cell 24: Method 21 - delete_all_dataset_items() - Demo only
print("=" * 60)
print("METHOD 21: delete_all_dataset_items()")
print("=" * 60)
print("Demonstrating delete_all_dataset_items()...")

# Note: This method requires specific item IDs to delete
# For demo purposes, we will use some example item IDs
example_item_ids = ["item-1", "item-2", "item-3"]
print(f"Attempting to delete items: {example_item_ids}")

try:
    delete_result = client.delete_all_dataset_items(DATASET_SLUG, example_item_ids)
    print(f"✓ Deletion result: {delete_result}")
except Exception as e:
    print(f"⚠ Deletion failed (expected if items do not exist): {type(e).__name__}: {e}")

print("\nNote: delete_all_dataset_items() requires item_ids as second parameter (required)")
print("The method will throw ValidationError if no item_ids are provided")


## Scorer Results Methods (6 methods)

Demonstrating all scorer results-related API methods.


In [ ]:
# Cell 25
# Re-add items for scorer demo
add_items_result = client.add_dataset_items(
    dataset_slug=DATASET_SLUG,
    items=dataset_items
)
print(f"✓ Re-added {len(dataset_items)} items for scorer results demo")


In [ ]:
# Cell 26: Method 22 - create_scorer_result()
print("=" * 60)
print("METHOD 22: create_scorer_result()")
print("=" * 60)
print("\nCreating a single scorer result...\n")

# Get first item for scoring
items_list = client.list_dataset_items(DATASET_SLUG, limit=1)
first_item_id = items_list["items"][0].get("id", items_list["items"][0].get("item_key", "trace_001"))

scorer_result_data = {
    "datasetSlug": DATASET_SLUG,
    "itemId": first_item_id,
    "scorerId": SCORER_ID,
    "score": 0.95,
    "metadata": {
        "confidence": 0.92,
        "model": "demo-scorer-v1"
    },
    "details": {
        "quality_aspects": {
            "completeness": 0.98,
            "accuracy": 0.94,
            "relevance": 0.93
        }
    }
}

scorer_result = client.create_scorer_result(scorer_result_data, organizationSlug=ORGANIZATION_SLUG)
print("✓ Scorer result created successfully")
pprint(scorer_result)


In [ ]:
# Cell 27: Method 23 - create_scorer_results_batch()
print("=" * 60)
print("METHOD 23: create_scorer_results_batch()")
print("=" * 60)
print("\nCreating scorer results in batch...\n")

# Get all items for batch scoring
items_list = client.list_dataset_items(DATASET_SLUG, limit=50)

batch_results = []
for i, item in enumerate(items_list["items"]):
    item_id = item.get("id", item.get("item_key"))
    # Create varied scores for demonstration
    score = 0.75 + (i * 0.02)  # Scores from 0.75 to 0.93
    if score > 1.0:
        score = 0.95

    result = {
        "datasetSlug": DATASET_SLUG,
        "itemId": item_id,
        "scorerId": f"{SCORER_ID}-batch",
        "score": round(score, 2),
        "metadata": {
            "batch_index": i,
            "processing_time_ms": 120 + (i * 10)
        }
    }
    batch_results.append(result)

batch_create_result = client.create_scorer_results_batch(batch_results, organizationSlug=ORGANIZATION_SLUG)
print(f"✓ Created {len(batch_results)} scorer results in batch")
pprint(batch_create_result)


In [ ]:
# Cell 28: Method 24 - list_scorer_results()
print("=" * 60)
print("METHOD 24: list_scorer_results()")
print("=" * 60)
print("\nListing scorer results...\n")

scorer_results_list = client.list_scorer_results(
    organizationSlug=ORGANIZATION_SLUG,
    datasetSlug=DATASET_SLUG,
    limit=100,
    offset=0
)

pprint(scorer_results_list)

if "results" in scorer_results_list:
    print(f"\n✓ Found {len(scorer_results_list['results'])} scorer results")


In [ ]:
# Cell 29: Method 25 - get_scorer_result()
print("=" * 60)
print("METHOD 25: get_scorer_result()")
print("=" * 60)
print("\nFetching specific scorer result...\n")

scorer_result_detail = client.get_scorer_result(
    dataset_slug=DATASET_SLUG,
    item_id=first_item_id,
    scorer_id=SCORER_ID,
    organizationSlug=ORGANIZATION_SLUG
)

pprint(scorer_result_detail)


In [ ]:
# Cell 30: Method 26 - update_scorer_result()
print("=" * 60)
print("METHOD 26: update_scorer_result()")
print("=" * 60)
print("\nUpdating scorer result...\n")

update_data = {
    "score": 0.98,
    "metadata": {
        "confidence": 0.97,
        "model": "demo-scorer-v2",
        "updated": True
    },
    "details": {
        "quality_aspects": {
            "completeness": 0.99,
            "accuracy": 0.98,
            "relevance": 0.97
        },
        "improvement": "Enhanced scoring algorithm applied"
    }
}

updated_scorer_result = client.update_scorer_result(
    dataset_slug=DATASET_SLUG,
    item_id=first_item_id,
    scorer_id=SCORER_ID,
    result_data=update_data,
    organizationSlug=ORGANIZATION_SLUG
)

print("✓ Scorer result updated successfully")
pprint(updated_scorer_result)


In [ ]:
# Cell 31: Method 27 - delete_scorer_result()
print("=" * 60)
print("METHOD 27: delete_scorer_result()")
print("=" * 60)
print("\nDeleting specific scorer result...\n")

delete_scorer_result = client.delete_scorer_result(
    dataset_slug=DATASET_SLUG,
    item_id=first_item_id,
    scorer_id=SCORER_ID,
    organizationSlug=ORGANIZATION_SLUG
)

print("✓ Scorer result deleted successfully")
pprint(delete_scorer_result)


## Cleanup


In [ ]:
# Cell 32: Method 28 - delete_dataset()
print("=" * 60)
print("METHOD 28: delete_dataset()")
print("=" * 60)
print(f"\nDeleting dataset: {DATASET_SLUG}\n")

delete_dataset_result = client.delete_dataset(DATASET_SLUG)
print(f"✓ Dataset {DATASET_SLUG} deleted successfully")
pprint(delete_dataset_result)


## Summary

### Congratulations! 🎉

You have successfully demonstrated all **28 methods** of the Noveum Platform API Client!

#### Methods Demonstrated:

**Trace Methods (7)**
1. `get_connection_status()` - Check API connection
2. `get_directory_tree()` - Get organization structure
3. `ingest_trace()` - Ingest single trace
4. `ingest_traces()` - Batch ingest traces
5. `query_traces()` - Query traces with filters
6. `get_trace()` - Get specific trace
7. `get_trace_spans()` - Get trace spans

**Dataset Methods (15)**
8. `create_dataset()` - Create new dataset
9. `list_datasets()` - List all datasets
10. `get_dataset()` - Get specific dataset
11. `update_dataset()` - Update dataset metadata
12. `create_dataset_version()` - Create dataset version
13. `list_dataset_versions()` - List dataset versions
14. `get_dataset_version()` - Get specific version
15. `get_dataset_versions_diff()` - Get diff between current and next release
16. `add_dataset_items()` - Add items to dataset
17. `list_dataset_items()` - List dataset items
18. `get_dataset_item()` - Get specific item
19. `publish_dataset_version()` - Publish version
20. `delete_dataset_item()` - Delete specific item
21. `delete_all_dataset_items()` - Delete all items
22. `delete_dataset()` - Delete dataset

**Scorer Results Methods (6)**
23. `create_scorer_result()` - Create single result
24. `create_scorer_results_batch()` - Batch create results
25. `list_scorer_results()` - List scorer results
26. `get_scorer_result()` - Get specific result
27. `update_scorer_result()` - Update result
28. `delete_scorer_result()` - Delete result

### Next Steps
- Customize the configuration variables at the top to use with your own data
- Explore different filter options in query methods
- Integrate these methods into your own workflows
- Check the [API documentation](../src/novaeval/noveum_platform/README.md) for more details
